In [3]:
import gco

/Users/admin/EE5371/CA2/.venv/lib/python3.8/site-packages/gco/__init__.py:7: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, name):
